In [1]:
#pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [2]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should return "NVIDIA GeForce GTX 1650"

True
NVIDIA GeForce GTX 1650


In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'  #to use batch
#device = 'cpu'
print(device)

block_size = 32# Reduced from 64
batch_size = 128  # Reduced from 128
max_iters = 300  # Reduced from 3000
learning_rate = 3e-4
eval_iters = 100  # Reduced from 100
#eval_interval = 50  # Reduced from 500
n_embd = 384  # Reduced from 384
n_head = 4  # Reduced from 4
n_layer = 4  # Reduced from 4
dropout = 0.2  # Kept the same

cuda


In [5]:
chars = ""
with open('vocab.txt','r', encoding = 'utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)

In [6]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]   #generator for-loops
decode = lambda l: ''.join([int_to_string[i] for i in l])



In [7]:
def get_random_chunk(split):
    filename = "output_train.txt" if split == 'train' else "output_val.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data


def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [8]:
@torch.no_grad() #make sure that pythorch does not use gradient at all here reduce computations
def estimate_loss():
    out = {}
    model.eval()
    for split in['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() #puts model in training mode
    return out

In [13]:
class Head(nn.Module):
    # one head of multi attention

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # inputs of size (batch, time-step, channels)
        # outputs of size(batch, time-step, head size)
        B, T, C = x.shape
        k = self.key(x) # (B,T,hs)
        q = self.query(x) #(B,T, hs)

        #compare attention scores(affinitites)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T ,T)
        wei = F.softmax(wei, dim = -1) #(B, T, T)
        wei = self.dropout(wei)
        # perform final weighted aggregation of the values
        v = self.value(x) # (B, T, hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

    

class MultiHeadAttention(nn.Module):
    # multiple heads of self-attention in parallel

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim= -1) # (B, T, F) -> (B, T, [h1,h1,h1,h1,h2,h2,h2,h2,h3,h3,h3,h3]
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    # a simple linear layer followed by a non-linearity
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout), # makes certain neurons 0, to prevent overfitting
        )
    def forward(self, x):
            return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x
        
 
class GPTLanguageModel(nn.Module):  
   
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) #block is the sequence size, each of length n_embd
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range (n_layer)])

        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size) # final linear layer from the previous sequential decoder outputs, and transform into softmax workable

        self.apply(self.__init__weights)

    
    def __init__weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean = 0.0, std = 0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std= 0.02)

            
    def forward(self, index, targets=None):
        #logits = self.token_embedding_table(index) #probability dist of what we wanna predict
        B, T = index.shape 

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) #(B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device)) # (T,C) see how long is T, here 8 and give each of the n_embedding vector for each of hte indicesf
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B,T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape #time as the sequence of integers.we do not know the next token, thats why we call it time
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
        
    def generate(self, index, max_new_tokens):
         for _ in range(max_new_tokens):
             logits, loss = self.forward(index)
             logits = logits[:, -1, :] #becomes (B,C)
             probs = F.softmax(logits, dim=-1) #(B,C)            
             index_next = torch.multinomial(probs, num_samples=1) #(B,1)         
             index = torch.cat((index, index_next), dim=1) #(b, T+1)
         return index

model = GPTLanguageModel(vocab_size)
#torch.cuda.empty_cache()
#device = torch.device("cpu")
m = model.to(device)
print(device)

#context = torch.zeros((1,1), dtype= torch.long, device=device)
#generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
#print(generated_chars)

cuda


This refers to a custom class (likely defined elsewhere in your code) that represents a single decoder block in the Transformer.

**Each Block typically contains:**

*Multi-Head Attention:* To compute attention over the input sequence.

*Feed-Forward Network (FFN):* To process the output of the attention mechanism.

*Residual Connections and Layer Normalization:* To stabilize training and improve gradient flow.

The **'*'** operator unpacks the list of blocks into individual arguments for nn.Sequential.

In [16]:
import mmap
import random
import pickle

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

#standard training loop architecture for this model
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")  #print every certain number of iter
   
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) #by default pytorch will accumulate the gradients over time by adding them and by 0 we make sure we avoid it
    loss.backward()
    optimizer.step() #let gradient descent work its magic  
print(loss.item())

with open('model_01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

KeyboardInterrupt: 